In [9]:
import pandas as pd 
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

**Research**

- https://www.finder.com/uk/working-from-home-statistics   
'If those with remote compatible jobs worked at home half the time it could result in saving 54 million tons of greenhouse gas – the equivalent of taking 10 million cars off the road.'

# Mobility Data

In [162]:
mobility = pd.read_csv('data/COVID-19-transport-use-statistics.csv')
mobility

,Date,Cars,Light Commercial Vehicles,Heavy Goods Vehicles,All motor vehicles,National Rail,Transport for London Tube,Transport for London Bus,Bus (excl. London),Cycling
0,01/03/2020,1.03,1.11,1.08,1.04,0.97,1.04,1.02,NaN,NaN
1,02/03/2020,1.02,1.06,1.03,1.03,0.94,0.95,0.97,NaN,NaN
2,03/03/2020,1.01,1.05,1.02,1.02,0.95,0.95,0.96,NaN,NaN
3,04/03/2020,1.01,1.04,1.03,1.01,0.95,0.95,0.97,NaN,NaN
4,05/03/2020,1.00,1.03,1.02,1.00,0.97,0.92,0.92,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
361,25/02/2021,0.64,0.83,1.01,0.70,NaN,0.18,0.36,0.30,1.13
362,26/02/2021,0.68,0.85,1.00,0.73,NaN,0.20,0.39,0.32,1.17
363,27/02/2021,0.66,0.83,1.06,0.70,NaN,0.21,0.44,0.31,NaN
364,28/02/2021,0.65,0.81,1.06,0.69,NaN,0.18,0.39,0.34,NaN


In [163]:
fig = px.line(mobility, x=mobility.Date, y=mobility.drop(['Date'], axis=1).columns,
              title='Mobility during covid')
fig.update_xaxes(rangeslider_visible = True)

# add baseline
fig.add_shape(type="line",
        x0=mobility.index.min(),
        y0=1,
        x1=mobility.index.max(),
        y1=1,
    name = 'baseline',
    line=dict(color='Red'))

fig.update_layout(margin={"r":0,"t":50,"l":0,"b":30})
fig.show()

In [164]:
# convert strings to datetime
for i in range(len(mobility['Date'])):
    mobility['Date'][i] = datetime.strptime(mobility['Date'][i], '%d/%m/%Y')

<ipython-input-164-4a5d52eedf89>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [167]:
cars_covid  = mobility[(mobility['Date'] > datetime(2020,4,1)) & (mobility['Date'] < datetime(2020,4,30)) ]['Cars']
lcv_covid = mobility[(mobility['Date'] > datetime(2020,4,1)) & (mobility['Date'] < datetime(2020,4,30))]['Light Commercial Vehicles']
hgv_covid = mobility[(mobility['Date'] > datetime(2020,4,1)) & (mobility['Date'] < datetime(2020,4,30))]['Heavy Goods Vehicles']
others_covid = mobility[(mobility['Date'] > datetime(2020,4,1)) & (mobility['Date'] < datetime(2020,4,30))]['All motor vehicles']

sum_vehicles = (cars_covid+lcv_covid+hgv_covid+others_covid)/4

print('Cars mobility mean')
print(np.mean(cars_covid), '\n')

print('Overal mean')
print(np.mean(sum_vehicles))

Cars mobility mean
0.3307142857142858 

Overal mean
0.4283035714285714


In [168]:
# percentage change from baseline
change_cars = (np.mean(cars_covid)-1)/1
print('Car mobility decrese:' , change_cars)

change_all = (np.mean(sum_vehicles)-1)/1
print('All vehicles mobility decrese:' , change_all)


Car mobility decrese: -0.6692857142857143
All vehicles mobility decrese: -0.5716964285714285


# Carbon Emission

https://carbonmonitor.org  
data filtered for UK, for Ground Transport sector from 1st January 2019 to 31st December 2020

In [93]:
co2 = pd.read_csv('data/carbon-monitor-UK.csv')
co2

,date,sector,MtCO2 per day
0,01/01/2019,Ground Transport,0.240374
1,02/01/2019,Ground Transport,0.233257
2,03/01/2019,Ground Transport,0.298681
3,04/01/2019,Ground Transport,0.301332
4,05/01/2019,Ground Transport,0.257965
...,...,...,...
726,27/12/2020,Ground Transport,0.242077
727,28/12/2020,Ground Transport,0.258263
728,29/12/2020,Ground Transport,0.299490
729,30/12/2020,Ground Transport,0.300395


In [25]:
fig = px.line(co2, x=co2.date, y=co2.drop(['date', 'sector'], axis=1).columns,
              title='Carbon emission during covid')
fig.update_xaxes(rangeslider_visible = True)

fig.update_layout(margin={"r":0,"t":50,"l":0,"b":30})
fig.show()

In [94]:
# convert strings to datetime
for i in range(len(co2['date'])):
    co2['date'][i] = datetime.strptime(co2['date'][i], '%d/%m/%Y')

<ipython-input-94-b2509f916dfd>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [173]:
# average pre covid
old_co2 = np.mean(co2[co2['date'] < datetime(2020, 3, 1)]['MtCO2 per day'])
old_co2

0.31364421374117674

In [176]:
# average during april
new_co2 = np.mean(co2[(co2['date'] > datetime(2020, 4, 1)) & (co2['date'] < datetime(2020, 4, 30))]['MtCO2 per day'])
new_co2

0.1827151192857143

In [177]:
(new_co2-old_co2)/old_co2

-0.41744463541580534

# Homeworking

In [285]:
homeworking_ons = pd.read_csv('data/homework_ons.csv')


fig = px.line(homeworking_ons, x=homeworking_ons.Date, y=homeworking_ons.drop(['Date'], axis=1).columns,
              title='Homeworking')
fig.update_xaxes(rangeslider_visible = True)

fig.update_layout(margin={"r":0,"t":50,"l":0,"b":30})
fig.show()

In [147]:
homeworking1 = pd.read_csv('data/homeworking.tsv')
homeworking1.iloc[:, : 5]
name = 'geo\\time\t2019 \t2018 \t2017 \t2016 \t2015 \t2014 \t2013 \t2012 \t2011 \t2010 \t2009 \t2008 \t2007 \t2006 \t2005 \t2004 \t2003 \t2002 \t2001 \t2000 \t1999 \t1998 \t1997 \t1996 \t1995 \t1994 \t1993 \t1992 '
geo = []
for i in range(len(homeworking1[name])):
    geo.append(homeworking1[name][i][0:2])

In [152]:
homeworking2 = pd.read_csv('data/homeworking.tsv', delimiter='\t')
homeworking2.iloc[:,1:]

,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,2001,2000,1999,1998,1997,1996,1995,1994,1993,1992
0,: u,: u,: u,: u,: u,: u,: u,: u,: u,: u,...,: c,: u,: c,: u,: u,: u,: u,:,:,:
1,: c,: c,: bc,: u,: c,: c,: c,:,: bu,: c,...,: b,:,: bu,: c,:,:,: u,: c,: c,: c
2,:,: c,:,:,:,: c,: c,: c,: bc,:,...,: b,:,:,:,:,:,:,:,:,:
3,80.2 u,85.9 u,76.6 u,: u,69.6 u,62.3 u,71.1 u,88.4 u,61.3 u,84.1 bu,...,: u,67.3 u,56.3 u,56.4 u,75.7 u,: u,:,:,:,:
4,: c,: c,:,:,:,: c,: c,: c,: c,: u,...,: c,:,: c,:,:,:,:,:,:,:
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70556,: u,:,:,: u,: u,: bu,: u,:,: c,:,...,:,:,:,:,:,:,:,:,:,:
70557,:,:,:,: c,: u,: c,:,:,:,: c,...,:,: c,:,:,:,:,:,:,:,:
70558,:,:,:,:,:,:,:,:,:,:,...,:,:,: b,:,:,:,:,:,:,:
70559,:,:,:,:,:,: b,:,:,:,: u,...,:,:,:,:,:,:,:,:,:,:


In [159]:
homeworking = homeworking1.iloc[:, : 5]
homeworking['geo'] = geo
homeworking = pd.concat([homeworking, homeworking2.iloc[:,1:]], axis=1, join="inner")
homeworking[homeworking['geo']=='UK']

,unit,sex,frequenc,age,wstatus,geo,2019,2018,2017,2016,...,2001,2000,1999,1998,1997,1996,1995,1994,1993,1992
36,PC,F,NVR,Y15-19,CFAM,UK,: c,: c,: c,: c,...,: c,: c,: bu,: u,: u,: u,: u,: u,: u,: u
75,PC,F,NVR,Y15-19,EMP,UK,96.6,97.7,96.5,97.7,...,98.6 u,97.6 u,96.4 bu,97.2,96.7 u,96.5 u,96.6 u,96.1 u,94.3 u,94.1 u
114,PC,F,NVR,Y15-19,NCFAM,UK,96.7,97.7,96.5,97.9,...,98.6,97.6 u,96.7 b,97.5,96.8 u,96.7 u,96.6 u,96.1 u,94.3 u,94.4 u
127,PC,F,NVR,Y15-19,NRP,UK,:,:,:,:,...,:,:,: b,:,:,:,:,:,:,:
166,PC,F,NVR,Y15-19,NSAL,UK,: u,: u,: u,85.9 u,...,: u,: u,: bu,: u,: u,: u,: u,80.6 u,: u,: u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70434,PC,T,USU,Y_GE75,NSAL,UK,35.6,38.8,34.2,39.6,...,: u,44.9 u,: bu,: u,: u,: u,: u,: u,: u,33.9 u
70465,PC,T,USU,Y_GE75,SAL,UK,19.1 u,: u,: u,22.2 u,...,: u,: u,: bu,: u,: u,: u,: u,: u,: u,: u
70497,PC,T,USU,Y_GE75,SELF,UK,39.0,39.5,38.2,38.6,...,: u,47.8 u,: bu,: u,: u,: u,: u,: u,: u,: u
70529,PC,T,USU,Y_GE75,SELF_NS,UK,43.8,42.1,42.9,44.0,...,: u,51.3 u,: bu,: u,: u,: u,: u,: u,: u,: u


# Lin model

In [181]:
import statsmodels.api as sm

In [272]:
co2_df = co2[['MtCO2 per day', 'date']].rename(columns={'date':'Date'})
mobility_df = mobility[['Date','Cars', 'Light Commercial Vehicles', 'Heavy Goods Vehicles', 'All motor vehicles']]
df = mobility_df.merge(co2_df , on='Date')
y = df['MtCO2 per day']
X = df.drop(['Date','MtCO2 per day'], axis = 1)


'''
X_cars = (X['Cars']-np.mean(X['Cars']))/np.std(X['Cars'])

# standardize

y = (y-np.mean(y))/np.std(y)

for i in range(X.shape[1]):
    X.iloc[:,[i]] = (X.iloc[:,[i]]-np.mean(X.iloc[:,[i]]))/np.std(X.iloc[:,[i]])'''

"\nX_cars = (X['Cars']-np.mean(X['Cars']))/np.std(X['Cars'])\n\n# standardize\n\ny = (y-np.mean(y))/np.std(y)\n\nfor i in range(X.shape[1]):\n    X.iloc[:,[i]] = (X.iloc[:,[i]]-np.mean(X.iloc[:,[i]]))/np.std(X.iloc[:,[i]])"

In [263]:
'''from sklearn.preprocessing import StandardScaler

# the scaler object (model)
scaler = StandardScaler()
# fit and transform the data
X = scaler.fit_transform(X) 

print(X)'''

'from sklearn.preprocessing import StandardScaler\n\n# the scaler object (model)\nscaler = StandardScaler()\n# fit and transform the data\nX = scaler.fit_transform(X) \n\nprint(X)'

In [277]:
X = sm.add_constant(X)

model = sm.OLS(y, X['Cars']).fit()
predictions = model.predict(X['Cars']) 

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          MtCO2 per day   R-squared (uncentered):                   0.968
Model:                            OLS   Adj. R-squared (uncentered):              0.968
Method:                 Least Squares   F-statistic:                              9261.
Date:                Sat, 06 Mar 2021   Prob (F-statistic):                   2.88e-230
Time:                        18:42:01   Log-Likelihood:                          479.85
No. Observations:                 306   AIC:                                     -957.7
Df Residuals:                     305   BIC:                                     -954.0
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Cars           0.3690      0.004     96.236      0.000       0.361       0.377
==============================================================================
Omnibus:                       18.216   Durbin-Watson:                   0.601
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               19.871
Skew:                          -0.616   Prob(JB):                     4.84e-05
Kurtosis:                       3.203   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [278]:
model.params

Cars    0.36903
dtype: float64

In [280]:
0.3690 * 33.07142857142858 

12.203357142857145